In [1]:
import numpy as np
import os
import pickle as p
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import sys
import slam.io as sio
import networkx as nx
import tools.graph_visu as gv
import tools.graph_processing as gp
import matplotlib.pyplot as plt
import random
import plotly.express as px
import plotly.figure_factory as ff
import trimesh
import slam.topology as stop
from sphere import *
import random

In [2]:
def generate_sphere_random_sampling(vertex_number=100, radius=1.0):
    """
    generate a sphere with random sampling
    :param vertex_number: number of vertices in the output spherical mesh
    :param radius: radius of the output sphere
    :return:
    """
    coords = np.zeros((vertex_number, 3))
    for i in range(vertex_number):
        M = np.random.normal(size=(3, 3))
        Q, R = np.linalg.qr(M)
        coords[i, :] = Q[:, 0].transpose() * np.sign(R[0, 0])
    if radius != 1:
        coords = radius * coords
    return coords

In [3]:
def tri_from_hull(vertices):
    """
    compute faces from vertices using trimesh convex hull
    :param vertices: (n, 3) float
    :return:
    """
    mesh = trimesh.Trimesh(vertices=vertices, process=False)
    return mesh.convex_hull

In [4]:
# def generate_noisy_graph(original_graph, nb_vertices, sigma_noise_nodes = 1, sigma_noise_edges = 1, radius = 100):
    
#     # generate ground_truth permutation
#     ground_truth_permutation = np.random.permutation(nb_vertices)
#     #ground_truth_permutation = np.arange(nb_vertices)
    
#     # create a new graph
#     noisy_graph = nx.Graph()

#     # add the nodes (not very clean but it works fine and run in no time)
#     for node_to_add in range(len(ground_truth_permutation)):
#         for original_node, current_node in enumerate(ground_truth_permutation):
#             if current_node == node_to_add:
#                 print(original_node,node_to_add)
#                 noisy_coordinate = original_graph.nodes[original_node]["coord"] + \
#                     np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)

#                 # We project on the sphere
#                 noisy_coordinate = noisy_coordinate / np.linalg.norm(noisy_coordinate) * radius
#                 noisy_graph.add_node(node_to_add, coord = noisy_coordinate)
        
#     compute_noisy_edges = tri_from_hull(list(nx.get_node_attributes(noisy_graph,'coord').values())) # take all peturbated coord and comp conv hull.
#     adja = stop.edges_to_adjacency_matrix(compute_noisy_edges) # compute the new adjacency mat
#     edge_list = [tuple((u,v)) for u,v in zip(adja.nonzero()[0],adja.nonzero()[1])] # convert to edge list to add edge attributes.


#     # add the edges
#     for edge in edge_list:

#         # get the original and corresponding ends
#         #end_a_corresponding, end_b_corresponding = ground_truth_permutation[edge[0]], ground_truth_permutation[edge[1]]
#         coordinate_a, coordinate_b = noisy_graph.nodes[edge[0]]["coord"], noisy_graph.nodes[edge[1]]["coord"]

#         # calculate noisy geodesic distance
#         noisy_geodesic_dist = gp.compute_geodesic_distance_sphere(coordinate_a, coordinate_b, radius)

#         # Add the new edge to the graph
#         noisy_graph.add_edge(edge[0],edge[1], weight = 1.0, geodesic_distance = noisy_geodesic_dist)
    
#     return ground_truth_permutation, noisy_graph

In [5]:
# def generate_noisy_graph_2(original_graph, nb_vertices,nb_outliers,sigma_noise_nodes=1, sigma_noise_edges=1, radius=100):
#     # Perturbate the coordinates
#     # noisy_coord = [points+np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)
#     # 	for points in list(nx.get_node_attributes(original_graph,'coord').values())]

#     ground_truth_permutation = np.arange(nb_vertices)
#     # create a new graph
#     noisy_graph = nx.Graph()
#     # add the nodes (not very clean but it works fine and run in no time)
#     for node_to_add in range(len(ground_truth_permutation)):
#         for original_node, current_node in enumerate(ground_truth_permutation):
#             if current_node == node_to_add:
#                 # noisy_coordinate = original_graph.nodes[original_node]["coord"] + \
#                 # 	np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)

#                 # Sampling from Von Mises - Fisher distribution
#                 original_coord = original_graph.nodes[original_node]["coord"]
#                 mean_original = original_coord / np.linalg.norm(original_coord)  # convert to mean vector
#                 noisy_coordinate = Sphere().sample(1, distribution='vMF', mu=mean_original,
#                                                    kappa=sigma_noise_nodes).sample[0]

#                 #noisy_coordinate = noisy_coordinate / np.linalg.norm(noisy_coordinate) * radius
#                 noisy_coordinate  = noisy_coordinate * np.linalg.norm(original_coord)
#                 noisy_graph.add_node(node_to_add, coord=noisy_coordinate)
                
#     noisy_coord = list(nx.get_node_attributes(noisy_graph, 'coord').values())
                
#     # Add Outliers
#     if nb_outliers > 0:
#         print("nb_outliers: ",nb_outliers)
#         sphere_random_sampling = generate_sphere_random_sampling(vertex_number=nb_outliers, radius=radius)
#         # merge pertubated and outlier coordinates to add edges 
#         all_coord = noisy_coord + list(sphere_random_sampling)
#     else:
#         all_coord = noisy_coord

#     print("nb_outliers: ",nb_outliers)


#     compute_noisy_edges = tri_from_hull(all_coord)  # take all peturbated coord and comp conv hull.
#     adja = stop.edges_to_adjacency_matrix(compute_noisy_edges)  # compute the new adjacency mat.

#     # Extracting the ground-truth correspondence
#     ground_truth_permutation = []
#     for ar1 in compute_noisy_edges.vertices.view(np.ndarray):
#         index = 0
#         for ar2 in noisy_coord:
#             if np.mean(ar1) == np.mean(ar2):
#                 ground_truth_permutation.append(index)
#                 index += 1
#                 break
#             else:
#                 index += 1
#                 continue

#     print("Total Ground truth nodes: ", len(ground_truth_permutation))
#     print("Total number of nodes : ", len(compute_noisy_edges.vertices))

#     noisy_graph = nx.from_numpy_matrix(adja.todense())

#     node_attribute_dict = {}
#     for node in noisy_graph.nodes():
#         node_attribute_dict[node] = {"coord": np.array(compute_noisy_edges.vertices[node])}

#     nx.set_node_attributes(noisy_graph, node_attribute_dict)

#     nx.set_edge_attributes(noisy_graph, 1.0, name="weight")

#     edge_attribute_dict = {}
#     id_counter = 0  # useful for affinity matrix caculation
#     for edge in noisy_graph.edges:
#         # We calculate the geodesic distance
#         end_a = noisy_graph.nodes()[edge[0]]["coord"]
#         end_b = noisy_graph.nodes()[edge[1]]["coord"]
#         geodesic_dist = gp.compute_geodesic_distance_sphere(end_a, end_b, radius)

#         # add the information in the dictionnary
#         edge_attribute_dict[edge] = {"geodesic_distance": geodesic_dist, "id": id_counter}
#         id_counter += 1

#     # add the edge attributes to the graph
#     nx.set_edge_attributes(noisy_graph, edge_attribute_dict)

#     return np.array(ground_truth_permutation), noisy_graph

In [393]:
def generate_noisy_graph_3(original_graph, nb_vertices,nb_outliers,sigma_noise_nodes=1, sigma_noise_edges=1, radius=100):
    # Perturbate the coordinates
    
    noisy_coord = []
    key = [] 
    value = []
    
    for index in range(nb_vertices):
        
        # Sampling from Von Mises - Fisher distribution
        original_coord = original_graph.nodes[index]["coord"]
        mean_original = original_coord / np.linalg.norm(original_coord)  # convert to mean vector
        noisy_coordinate = Sphere().sample(1, distribution='vMF', mu=mean_original,
                                           kappa=sigma_noise_nodes).sample[0]

        noisy_coordinate  = noisy_coordinate * np.linalg.norm(original_coord)
        #print(noisy_coordinate)
        noisy_coord.append(noisy_coordinate)
                   
    # Add Outliers
    sphere_random_sampling = []
    if nb_outliers > 0:
        print("nb_outliers: ",nb_outliers)
        sphere_random_sampling = generate_sphere_random_sampling(vertex_number=nb_outliers, radius=radius)
        # merge pertubated and outlier coordinates to add edges
        all_coord = noisy_coord + list(sphere_random_sampling)
    else:
        all_coord = noisy_coord

    print("nb_outliers: ",nb_outliers)

    compute_noisy_edges = tri_from_hull(all_coord)  # take all peturbated coord and comp conv hull.
    adja = stop.edges_to_adjacency_matrix(compute_noisy_edges)  # compute the new adjacency mat.

    # Extracting the ground-truth correspondence

    noisy_graph = nx.from_numpy_matrix(adja.todense())
    
    print("all nodes: ",len(noisy_graph.nodes))

    node_attribute_dict = {}
    for node in noisy_graph.nodes():
        node_attribute_dict[node] = {"coord": np.array(compute_noisy_edges.vertices[node])}

    nx.set_node_attributes(noisy_graph, node_attribute_dict)

    nx.set_edge_attributes(noisy_graph, 1.0, name="weight")

    edge_attribute_dict = {}
    id_counter = 0  # useful for affinity matrix caculation
    for edge in noisy_graph.edges:
        # We calculate the geodesic distance
        end_a = noisy_graph.nodes()[edge[0]]["coord"]
        end_b = noisy_graph.nodes()[edge[1]]["coord"]
        geodesic_dist = gp.compute_geodesic_distance_sphere(end_a, end_b, radius)

        # add the information in the dictionnary
        edge_attribute_dict[edge] = {"geodesic_distance": geodesic_dist, "id": id_counter}
        id_counter += 1

    # add the edge attributes to the graph
    nx.set_edge_attributes(noisy_graph, edge_attribute_dict)
    
    ground_truth_permutation = []

    for ar1 in noisy_coord:
        for i in range(len(noisy_graph.nodes)):
            if np.mean(ar1) == np.mean(noisy_graph.nodes[i]['coord']):    
                if i >=nb_vertices:
                    key.append(i)
                    print("key",i)
                ground_truth_permutation.append(i)
                break
                
                
    for outlier in sphere_random_sampling:
        for i in range(len(noisy_graph.nodes)):
            if np.mean(noisy_graph.nodes[i]['coord']) == np.mean(outlier):
                if i<nb_vertices:
                    value.append(i)
                    print("value",i)
                    
    
                 
    if nb_outliers > 0 and len(key)!=0:
        index = 0
        for j in range(len(ground_truth_permutation)):
            if ground_truth_permutation[j] == key[index]:
                ground_truth_permutation[j] = value[index]
                index+=1
                if index == len(key):
                    break
                    
        key = key + value
        value = value + key
    
        mapping = dict(zip(key,value))
        print(mapping)

        noisy_graph = nx.relabel_nodes(noisy_graph, mapping) # Relabeling Nodes to keep ground-truth in len(#nodes).

    return ground_truth_permutation,noisy_graph,noisy_coord,sphere_random_sampling

In [407]:
ground_truth

[19, 6, 12, 13, 2, 8, 3, 11, 1, 15, 9, 18, 16, 14, 0, 4, 5, 10, 7, 17]

In [395]:
G = nx.read_gpickle("reference_0.gpickle")

In [396]:
original_coord =  list(nx.get_node_attributes(G,'coord').values())

In [397]:
# noisy_graph_old_coord = list(nx.get_node_attributes(noisy_graph_old,'coord').values())
# noisy_graph_old_coord[:5]

In [398]:
ground_truth, noisy_graph,noisy_coord,outliers = generate_noisy_graph_3(G,len(original_coord),10,10,0)

nb_outliers:  10
nb_outliers:  10
all nodes:  30
key 22
key 26
key 28
key 24
key 23
key 29
key 27
value 13
value 8
value 3
value 15
value 9
value 0
value 5
{22: 13, 26: 8, 28: 3, 24: 15, 23: 9, 29: 0, 27: 5, 13: 22, 8: 26, 3: 28, 15: 24, 9: 23, 0: 29, 5: 27}


In [399]:
len(ground_truth)

20

In [400]:
noisy_graph.nodes

NodeView((29, 1, 2, 28, 4, 27, 6, 7, 26, 23, 10, 11, 12, 22, 14, 24, 16, 17, 18, 19, 20, 21, 13, 9, 15, 25, 8, 5, 3, 0))

In [401]:
ground_truth

[19, 6, 12, 13, 2, 8, 3, 11, 1, 15, 9, 18, 16, 14, 0, 4, 5, 10, 7, 17]

In [402]:
index = 0
for i in ground_truth:
    if np.mean(noisy_graph.nodes[i]['coord'] == noisy_coord[index]):
        print("    match: ",noisy_graph.nodes[i]['coord'],noisy_coord[index])
    else:
        print("Not match: ",noisy_graph.nodes[i]['coord'],noisy_coord[index])
    index+=1
print("Total indexes", index)

    match:  [ 97.0114961   10.72561958 -21.76535573] [ 97.0114961   10.72561958 -21.76535573]
    match:  [-16.50730932 -95.82102031 -23.36323622] [-16.50730932 -95.82102031 -23.36323622]
    match:  [-48.9121704   74.06926908 -46.05803908] [-48.9121704   74.06926908 -46.05803908]
    match:  [-36.76963146   0.8223222  -92.99095649] [-36.76963146   0.8223222  -92.99095649]
    match:  [70.02354486 56.33102381 43.85794023] [70.02354486 56.33102381 43.85794023]
    match:  [-46.65995061 -79.1154977  -39.54221835] [-46.65995061 -79.1154977  -39.54221835]
    match:  [-12.96400909 -47.27044067  87.16329449] [-12.96400909 -47.27044067  87.16329449]
    match:  [-54.27250579  71.83164411  43.52826692] [-54.27250579  71.83164411  43.52826692]
    match:  [-42.90835419 -87.57768752  22.11383705] [-42.90835419 -87.57768752  22.11383705]
    match:  [ 35.28525505 -79.17178789 -49.86761252] [ 35.28525505 -79.17178789 -49.86761252]
    match:  [ 15.13235383 -64.28237478  75.09186481] [ 15.13235383

In [403]:
outliers

array([[ 36.19014513, -60.81998733, -70.64844327],
       [ -8.06574392, -92.24560988, -37.75832667],
       [ 26.49864801, -96.05052698,  -8.4922271 ],
       [ 18.98045238,  44.42569073, -87.55627008],
       [-71.11433319,  51.9061427 , -47.41839269],
       [ 41.21954146,  17.16894909,  89.47724062],
       [-72.11741004,  32.65028254, -61.09859424],
       [  7.13361937, -98.64339734,  14.78484482],
       [-81.31148116,  33.30849211, -47.73874092],
       [-48.21938621, -70.98311831,  51.34479242]])

In [404]:
outlier_index = []
for outlier in outliers:
    for i in range(len(noisy_graph.nodes)):
        if np.mean(noisy_graph.nodes[i]['coord']) == np.mean(outlier):
            print(i)
            outlier_index.append(i)

22
25
26
28
24
23
21
29
20
27


In [405]:
list(set(outlier_index).intersection(ground_truth))

[]

In [ ]:
compute_noisy_edges = tri_from_hull(noisy_coord_3+list(outlier_coords))

In [ ]:
ground_truth_sequence = []

coords_new = compute_noisy_edges.vertices.view(np.ndarray)

key = []
value = []

for ar1 in noisy_coord_3:
    for i in range(len(coords_new)):
        if np.mean(ar1) == np.mean(coords_new[i]):
            if i >=20:
                key.append(i)
                print("key:",i)
            ground_truth_sequence.append(i)
            
            
for outlier in outlier_coords:
    for i in range(len(coords_new)):
        if np.mean(coords_new[i]) == np.mean(outlier):
            if i<20:
                value.append(i)
                print("value:",i)
                
index = 0
for j in range(len(ground_truth_sequence)):
    if ground_truth_sequence[j] == key[index]:
        ground_truth_sequence[j] = value[index]
        index+=1
                
mapping = dict(zip(key,value))

noisy_graph_3 = nx.relabel_nodes(noisy_graph_3, mapping)

In [ ]:
ground_truth_sequence

In [ ]:
noisy_coord_3[19]

In [ ]:
noisy_graph_3.nodes[6]

In [ ]:
test_G.nodes[0]

In [ ]:
ground_truth_sequence

In [ ]:
test_G = noisy_graph_3.copy()

test_G = nx.relabel_nodes(test_G, mapping)